In [1]:
# print('hello chatbot')

In [2]:
# pip install jupyter ipywidgets --upgrade


In [ ]:
from langchain import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Pinecone
import pinecone
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.prompts import PromptTemplate
from langchain.llms import CTransformers
from sentence_transformers import SentenceTransformer

In [2]:
PINECONE_API_KEY = "pcsk_6vak7A_QZQEv68cSuCxJjXkuFDQ1Y9q9Dt4rFJ5nsXwr8Hm1BTBsKQU541hnCqSj5o3GiT"
PINECONE_API_ENV = "gcp-starter"

In [3]:
#Extract data from the PDF
def load_pdf(data):
    loader = DirectoryLoader(data,
                    glob="*.pdf",
                    loader_cls=PyPDFLoader)
    
    documents = loader.load()

    return documents

In [4]:
extracted_data = load_pdf(data="../data") 


In [5]:
#Create text chunks
def text_split(extracted_data):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size = 500, chunk_overlap = 20)
    text_chunks = text_splitter.split_documents(extracted_data)

    return text_chunks

In [ ]:
text_chunks = text_split(extracted_data)
print("length of my chunk:", len(text_chunks))

In [7]:
# text_chunks

In [8]:
#download embedding model
def download_hugging_face_embeddings():
    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    return embeddings

In [11]:
# pip install sentence_transformers

In [9]:
embeddings = download_hugging_face_embeddings()

In [ ]:
embeddings

In [ ]:
query_result = embeddings.embed_query("Hello world")
print("Length", len(query_result))

In [15]:
# query_result

In [13]:
#Initializing the Pinecone
pinecone.init(api_key=PINECONE_API_KEY,
              environment=PINECONE_API_ENV)

index_name="medicalbot"

In [ ]:
#Creating Embeddings for Each of The Text Chunks & storing
# docsearch=Pinecone.from_texts([t.page_content for t in text_chunks], embeddings, index_name=index_name)

In [ ]:
import pinecone
from tqdm import tqdm

# Assuming you have the extracted data and embeddings ready
def create_and_store_embeddings(text_chunks, embeddings, index_name):
    # Connect to the Pinecone index
    index = pinecone.Index(index_name)
    
    # Create a list to hold the data for upserting to Pinecone
    upsert_data = []
    
    # Iterate over the text chunks and generate embeddings for each
    for idx, chunk in enumerate(tqdm(text_chunks)):
        # Generate embedding for the chunk using the correct method
        embedding = embeddings.embed_documents([chunk.page_content])[0]  # embed_documents returns a list
        
        # Store the embedding along with a unique ID (e.g., index) and the original chunk text
        upsert_data.append((str(idx), embedding, {'text': chunk.page_content}))
    
    # Upsert the embeddings into Pinecone
    index.upsert(vectors=upsert_data)

# Call the function to create and store embeddings
create_and_store_embeddings(text_chunks=extracted_data, embeddings=embeddings, index_name=index_name)


In [ ]:
# docsearch=Pinecone.from_texts([t.page_content for t in text_chunks], embeddings, index_name=index_name)

In [ ]:
docsearch=Pinecone.from_existing_index(index_name=index_name, embeddings=embeddings)

query = "What are Allergies"

docs=docsearch.similarity_search(query, k=3)

print("Result", docs)

In [ ]:
prompt_template="""
Use the following pieces of information to answer the user's question.
If you don't know the answer, just say that you don't know, don't try to make up an answer.

Context: {context}
Question: {question}

Only return the helpful answer below and nothing else.
Helpful answer:
"""

In [ ]:
PROMPT=PromptTemplate(template=prompt_template, input_variables=["context", "question"])
chain_type_kwargs={"prompt": PROMPT}

In [ ]:
llm=CTransformers(model="model/llama-2-7b-chat.ggmlv3.q4_0.bin",
                  model_type="llama",
                  config={'max_new_tokens':512,
                          'temperature':0.8})

In [ ]:
qa=RetrievalQA.from_chain_type(
    llm=llm, 
    chain_type="stuff", 
    retriever=docsearch.as_retriever(search_kwargs={'k': 2}),
    return_source_documents=True, 
    chain_type_kwargs=chain_type_kwargs)

In [ ]:
while True:
    user_input=input(f"Input Prompt:")
    result=qa({"query": user_input})
    print("Response : ", result["result"])